In [1]:
from Bio import SeqIO

def find_type(gb_file_name):
    types = []
    for rec in SeqIO.parse(gb_file_name, 'gb'):
#         print(rec.features)
        for feature in rec.features:
            if feature.type == "cand_cluster":
                types.append(feature.qualifiers['product'])
#                 print(types) 
    return types
                                
test=find_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_062771000.1.region001.gbk')
# test2=find_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_088907636.1.region001.gbk')
# test3=find_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_073559068.1.region001.gbk')
print(test)

[['NRPS', 'T1PKS', 'PKS-like', 'butyrolactone'], ['NRPS', 'T1PKS'], ['PKS-like'], ['butyrolactone'], ['butyrolactone']]


In [2]:
# FInd CAT domæne
from Bio import SeqIO
def find_cat_and_type(gb_file_name):
    BGC = []
    CATs = []
    KRDH = 0
    monomer = []
    types = []
    typ = False
    ser = False
    CAT_and_KRDH = False
    for rec in SeqIO.parse(gb_file_name, 'gb'):
        for feature in rec.features:
            if feature.type == "aSModule":
                BGC.append(feature.qualifiers['domains'])
                if 'monomer_pairings' in feature.qualifiers.keys():
                    monomer.append(feature.qualifiers['monomer_pairings'])
                else:
                    monomer.append('NaN')
                types.append(''.join(feature.qualifiers['type']))   
    for module in range(len(BGC)):
#         print(module)
        for domain in range(len(BGC[module])):
            if 'DH' in BGC[module][domain] and 'KR' in BGC[module][domain-1]:
                KRDH = int(module)
            if 'PCP' in BGC[module][domain]:
#                 print(len(BGC[module]))
                if 'AMP' in BGC[module][domain-1] and 'Condensation' in BGC[module][domain-2]:
                    CATs.append(int(module))
                elif len(BGC[module]) >3:
                    if 'AMP' in BGC[module][domain-2] and 'Condensation' in BGC[module][domain-3]:
                        CATs.append(int(module))

    CATs=sorted(CATs, reverse=True)
    for CAT in CATs:
        if KRDH-CAT <=2 and KRDH-CAT > 0:
            CAT_and_KRDH = 'before KR DH'
            if 'ser' in ''.join(monomer[CAT]):
                ser = 'serine'
        elif KRDH-CAT == -1:
            CAT_and_KRDH = 'after KR DH'
                        
#                         print(i)
    return CAT_and_KRDH,ser
#                 types.append(''.join(feature.qualifiers['type']))
# test=find_cat_and_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\MIBiG_ny_database\BGC0001332.gbk')
# test2=find_cat_and_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_099279319.1.region001.gbk')
# test3=find_cat_and_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_093723927.1.region001.gbk')
test4 = find_cat_and_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_101419868.1.region001.gbk')
print(test4)

('before KR DH', 'serine')


In [3]:
def make_data_lst_of_lst(directory):
    lst_of_lst=[]
    import os
    for filename in os.listdir(directory):
        name=str(filename[:-4])
        gb_file_name=r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\{}'.format(filename)
        typ=find_type(gb_file_name)
        CAT_and_KRDH,ser = find_cat_and_type(gb_file_name)
        lst_of_lst.append([name,typ,CAT_and_KRDH,ser])
    return lst_of_lst

tester=make_data_lst_of_lst(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp')

In [4]:
print(tester)

[['AGM05849.1.region001', [['NRPS', 'T1PKS']], 'after KR DH', False], ['AKF07185.1.region001', [['NRPS-like', 'bacteriocin', 'T1PKS'], ['NRPS-like'], ['bacteriocin'], ['T1PKS']], False, False], ['AKF80291.1.region001', [['T1PKS', 'NRPS']], 'after KR DH', False], ['AKH87109.1.region001', [['NRPS', 'T1PKS', 'PKS-like'], ['NRPS', 'T1PKS'], ['T1PKS', 'PKS-like']], 'before KR DH', 'serine'], ['APY90477.1.region001', [['NRPS', 'T1PKS', 'PKS-like', 'butyrolactone'], ['NRPS', 'T1PKS'], ['PKS-like'], ['butyrolactone'], ['butyrolactone']], 'before KR DH', 'serine'], ['BAU88116.1.region001', [['NRPS', 'T1PKS']], 'before KR DH', 'serine'], ['BAY73753.1.region001', [['NRPS', 'T1PKS']], 'after KR DH', False], ['BAY84413.1.region001', [['NRPS', 'T1PKS'], ['NRPS', 'T1PKS'], ['NRPS']], False, False], ['CCK25062.1.region001', [['NRPS', 'T1PKS']], 'before KR DH', 'serine'], ['CDZ24035.1.region001', [['lassopeptide', 'NRPS', 'transAT-PKS', 'T1PKS'], ['lassopeptide'], ['NRPS', 'transAT-PKS', 'T1PKS']], 'be

In [5]:
import pandas as pd
df=pd.DataFrame(tester, columns=['name','BGC type', 'CAT->KR DH', 'ser'])

In [9]:
# print(df['name'].nunique())

522


In [6]:
df.to_csv('BGC_type_and_CAT_blastp_new.txt', sep='\t')